Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
print(sales.shape)
sales.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
sales['date_block_num'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique()[:1]:
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    print('cur_shops ', cur_shops)
    print('cur_items ', cur_items)
    print('grid ', grid)

cur_shops  [59 25 24 23 19 22 18 21 28 27 29 26  4  6  2  3  7  0  1 16 15  8 10 14 13
 12 53 31 30 32 35 56 54 47 50 42 43 52 51 41 38 44 37 46 45]
cur_items  [22154  2552  2554 ...,   628   631   621]
grid  [array([[   59, 22154,     0],
       [   59,  2552,     0],
       [   59,  2554,     0],
       ..., 
       [   45,   628,     0],
       [   45,   631,     0],
       [   45,   621,     0]], dtype=int32)]


In [5]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [6]:
#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
print('gb', gb)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


gb         shop_id item_id date_block_num item_cnt_day
                                             target
0             0      30              1         31.0
1             0      31              1         11.0
2             0      32              0          6.0
3             0      32              1         10.0
4             0      33              0          3.0
5             0      33              1          3.0
6             0      35              0          1.0
7             0      35              1         14.0
8             0      36              1          1.0
9             0      40              1          1.0
10            0      42              1          1.0
11            0      43              0          1.0
12            0      49              1          2.0
13            0      51              0          2.0
14            0      51              1          3.0
15            0      57              1          1.0
16            0      59              1          1.0
17       

In [7]:
#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]

gb.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target'], dtype='object')

In [8]:
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [9]:
# all_data.target.value_counts()

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [10]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()
item_id_target_mean.head()

item_id
0    0.020000
1    0.023810
2    0.019802
3    0.019802
4    0.020000
Name: target, dtype: float64

In [11]:
# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
all_data[all_data.item_id ==0].head()

,shop_id,item_id,date_block_num,target,item_target_enc
7170477,2,0,20,0.0,0.02
7176548,3,0,20,0.0,0.02
7158335,4,0,20,0.0,0.02
7152264,5,0,20,0.0,0.02
7164406,6,0,20,0.0,0.02


In [12]:
# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values

In [13]:
np.corrcoef(all_data['target'].values, encoded_feature)

array([[ 1.       ,  0.4830387],
       [ 0.4830387,  1.       ]])

In [14]:
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


#### Method 2

In [15]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [16]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [18]:
all_data['item_id_mean_target'] = -1
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id_mean_target
139255,0,19,0,0.0,0.022222,-1
141495,0,27,0,0.0,0.056834,-1
144968,0,28,0,0.0,0.141176,-1
142661,0,29,0,0.0,0.037383,-1
138947,0,32,0,6.0,1.319042,-1


In [19]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = False)

for train_index, test_index in kf.split(all_data):
    X_tr, X_val = all_data.iloc[train_index], all_data.iloc[test_index]
    cols =['item_id']
    for col in cols:
        means = X_val[col].map(X_tr.groupby(col).target.mean())
        X_val[col+'_mean_target'] = means
#         print(X_val.head())
    all_data.iloc[test_index] = X_val
print(all_data.head())
prior = all_data.target.mean()
print('prior ', prior)
all_data['item_id_mean_target'].fillna(prior, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


        shop_id  item_id  date_block_num  target  item_target_enc  \
139255        0       19               0     0.0         0.022222   
141495        0       27               0     0.0         0.056834   
144968        0       28               0     0.0         0.141176   
142661        0       29               0     0.0         0.037383   
138947        0       32               0     6.0         1.319042   

        item_id_mean_target  
139255                  NaN  
141495             0.048523  
144968             0.142424  
142661             0.030303  
138947             0.894020  
prior  0.3342730567123426


In [20]:
# You will need to compute correlation like that
encoded_feature = all_data['item_id_mean_target'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.416459048853
Current answer for task KFold_scheme is: 0.416459048853


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [21]:
global_sum = all_data['target'].sum()
global_sum

3648206.0

In [22]:
n_objects = all_data.shape[0]

In [23]:
all_data['loo'] = (global_sum - all_data['target'])/(n_objects-1)

all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id_mean_target,loo
139255,0,19,0,0.0,0.022222,0.334273,0.334273
141495,0,27,0,0.0,0.056834,0.048523,0.334273
144968,0,28,0,0.0,0.141176,0.142424,0.334273
142661,0,29,0,0.0,0.037383,0.030303,0.334273
138947,0,32,0,6.0,1.319042,0.894020,0.334273


In [24]:
# YOUR CODE GOES HERE
encoded_feature = all_data['loo'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(np.corrcoef(all_data['target'].values, encoded_feature))
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

[[ 1. -1.]
 [-1.  1.]]
-1.0
Current answer for task Leave-one-out_scheme is: -1.0


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [25]:
item_id_counts = all_data.groupby('item_id').size()
all_data['nrows'] = all_data['item_id'].map(item_id_counts)
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id_mean_target,loo,nrows
139255,0,19,0,0.0,0.022222,0.334273,0.334273,45
141495,0,27,0,0.0,0.056834,0.048523,0.334273,739
144968,0,28,0,0.0,0.141176,0.142424,0.334273,595
142661,0,29,0,0.0,0.037383,0.030303,0.334273,321
138947,0,32,0,6.0,1.319042,0.894020,0.334273,1586


In [26]:
a = 100
globalmean = 0.3343
all_data['smoothing'] = (all_data['item_target_enc'] * all_data['nrows'] + globalmean * a)/(all_data['nrows'] + a)


In [27]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id_mean_target,loo,nrows,smoothing
139255,0,19,0,0.0,0.022222,0.334273,0.334273,45,0.237448
141495,0,27,0,0.0,0.056834,0.048523,0.334273,739,0.089905
144968,0,28,0,0.0,0.141176,0.142424,0.334273,595,0.168964
142661,0,29,0,0.0,0.037383,0.030303,0.334273,321,0.107910
138947,0,32,0,6.0,1.319042,0.894020,0.334273,1586,1.260635


In [28]:
# YOUR CODE GOES HERE
encoded_feature = all_data['smoothing'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [29]:
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumcnt= all_data.groupby('item_id').cumcount()


In [30]:
all_data['expanding_mean'] = cumsum/cumcnt
all_data['expanding_mean'].fillna(0.3343, inplace=True) 
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id_mean_target,loo,nrows,smoothing,expanding_mean
139255,0,19,0,0.0,0.022222,0.334273,0.334273,45,0.237448,0.3343
141495,0,27,0,0.0,0.056834,0.048523,0.334273,739,0.089905,0.3343
144968,0,28,0,0.0,0.141176,0.142424,0.334273,595,0.168964,0.3343
142661,0,29,0,0.0,0.037383,0.030303,0.334273,321,0.107910,0.3343
138947,0,32,0,6.0,1.319042,0.894020,0.334273,1586,1.260635,0.3343


In [31]:
# YOUR CODE GOES HERE

encoded_feature = all_data['expanding_mean'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.502524521108
Current answer for task Expanding_mean_scheme is: 0.502524521108


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [32]:
STUDENT_EMAIL = 'ciortanmadalina@gmail.com'
STUDENT_TOKEN = '4hK0HtY75jJyM4iW'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.416459048853
Task Leave-one-out_scheme: -1.0
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.502524521108


In [33]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
